In [2]:
import requests
import regex as re
import json
import time
import random
import networkx as nx

In [3]:
def open_file(filepath:str='cache.json')->dict:
    '''Opens a file'''
    cache_file = open(filepath, 'r')
    cache_contents = cache_file.read()
    artist_data = json.loads(cache_contents)
    cache_file.close()
    return artist_data

In [6]:
def load_full_network(playlist_link:str,random_link:str)->nx.Graph:
    playlist_network = open_file(playlist_link)
    random_network = open_file(random_link)
    # combine!!

    # playlist network doesn't have name key, my bad
    for artist in playlist_network.keys():
        playlist_network[artist]['name']=artist

    g = nx.Graph()
    for artist in playlist_network.keys():
        g.add_node(artist)
        colab = playlist_network[artist]['collaborators'].keys()
        for y in colab:
            if artist !=y:
                if y not in g.nodes:
                    g.add_node(y)
                g.add_edge(artist,y)

    for n in g.nodes():
        g.nodes[n]['source'] = 'playlist'
        if n in playlist_network.keys():
            for key in playlist_network[n].keys():
                g.nodes[n][key]= playlist_network[n][key]
        else:
            g.nodes[n]['in_playlist']=False
            g.nodes[n]['name']=n

    for artist in random_network.keys():
        if artist not in g.nodes:
            g.add_node(artist)
        colab = random_network[artist]['collaborators'].keys()
        for y in colab:
            if artist !=y:
                if y not in g.nodes:
                    g.add_node(y)
                g.add_edge(artist,y)

    for n in g.nodes():
        if n in random_network.keys():
            for key in random_network[n].keys():
                g.nodes[n][key]= random_network[n][key]
            g.nodes[n]['in_playlist']=False
            g.nodes[n]['name']=n
            if n not in playlist_network.keys():
                g.nodes[n]['source']='random'
    return g

In [7]:
g_10_26 = load_full_network('10_26_playlist_layer2.json','10_26_random_2layer.json')
g_11_01 = load_full_network('11_01_playlist_layer2.json','10_26_random_2layer.json')
g_11_08 = load_full_network('11_08_playlist_layer2.json','10_26_random_2layer.json')